<div style="text-align:right;">Justine Charrel - Master 1 Internship</div>

This script concatanate all the collocated data of ERA5 and MOSAiC.

### IMPORT

In [1]:
import glob
import xarray
from netCDF4 import Dataset as ncfile
import numpy as np
from netCDF4 import num2date
import matplotlib.dates as dates

In [2]:
years = ['2019','2020'] ; months = ['01','02','03','04','05','06','07','08','09','10','11','12']
paths = []
for year in years:
    for month in months :
        path = sorted(glob.glob('/lustre/storeB/users/justinec/master_internship/data/ERA5_MOSAiC/' + year + '/' + month + '/collocated_ERA5_MOSAiC_*.nc'))
        if path :
            for i in range(len(path)) :
                paths.append(path[i])

In [3]:
ds = xarray.open_mfdataset(paths,combine = 'nested', concat_dim="time")

In [4]:
ds

<xarray.Dataset>
Dimensions:          (pres: 29, time: 1399)
Coordinates:
  * pres             (pres) float64 300.0 325.0 350.0 ... 950.0 975.0 1e+03
Dimensions without coordinates: time
Data variables: (12/14)
    temp_era5        (time, pres) float64 dask.array<chunksize=(1, 29), meta=np.ndarray>
    temp_mosaic      (time, pres) float64 dask.array<chunksize=(1, 29), meta=np.ndarray>
    dewpoint_mosaic  (time, pres) float64 dask.array<chunksize=(1, 29), meta=np.ndarray>
    rh_era5          (time, pres) float64 dask.array<chunksize=(1, 29), meta=np.ndarray>
    rh_mosaic        (time, pres) float64 dask.array<chunksize=(1, 29), meta=np.ndarray>
    sh_era5          (time, pres) float64 dask.array<chunksize=(1, 29), meta=np.ndarray>
    ...               ...
    wspeed_mosaic    (time, pres) float64 dask.array<chunksize=(1, 29), meta=np.ndarray>
    wdir_era5        (time, pres) float64 dask.array<chunksize=(1, 29), meta=np.ndarray>
    wdir_mosaic      (time, pres) float64 dask.array<chunksize=(1, 29), meta=np.ndarray>
    lat_mosaic       (time, pres) float64 dask.array<chunksize=(1, 29), meta=np.ndarray>
    lon_mosaic       (time, pres) float64 dask.array<chunksize=(1, 29), meta=np.ndarray>
    time_mosaic      (time, pres) datetime64[ns] dask.array<chunksize=(1, 29), meta=np.ndarray>

In [5]:
path_output = '/lustre/storeB/users/justinec/master_internship/data/ERA5_MOSAiC/'
output_filename = path_output + '/concatenated_ERA5_MOSAiC.nc'

In [6]:
ds.to_netcdf(output_filename)

In [225]:
test = ncfile('/lustre/storeB/users/justinec/master_internship/data/ERA5_MOSAiC/concatenated_ERA5_MOSAiC.nc')

In [239]:
def convert_time(dataset) :
    dataset_time = dataset.variables['time_mosaic']
    t_unit = dataset_time.units
    t_cal = dataset_time.calendar
    dataset_nctime=[]; 
    dataset_nctime.append(num2date(dataset_time,units = t_unit,calendar = t_cal, only_use_python_datetimes=True, only_use_cftime_datetimes=False)) #datetime of type 2019-10-01 00:00:00
    #dates.date2num --> convert datetime objects to Matplotlib dates (better for compare ERA5 and MOSAiC times and for figures)
    #np.squeeze --> pass Matplotlib dates in column instead of in line
    dataset_nctimenum = np.squeeze(dates.date2num(dataset_nctime))
    return dataset_nctime,dataset_nctimenum,t_unit

In [240]:
mosaic_nctime, mosaic_nctimenum, t_unit = convert_time(test)